#UNZIP THE EDITED RRL

In [ ]:
import zipfile
import os

def unzip_file(zip_path, extract_to=None):
    if extract_to is None:
        extract_to = os.path.dirname(zip_path)

    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
            print(f"Files extracted to: {extract_to}")
    except zipfile.BadZipFile:
        print("Error: The file is not a zip file or it is corrupted.")
    except Exception as e:
        print(f"An error occurred: {e}")
zip_path = '/content/rrl.zip'
unzip_file(zip_path)

#PREPROCESS THE DATA FOR RRL

In [ ]:
import numpy as np
import pandas as pd

# Load the data
data_path = "/content/data_refined_new.csv"
data = pd.read_csv(data_path)

processed_data = data.copy()

for column in processed_data.columns[:-1]:
    processed_data[column] = processed_data[column].apply(lambda x: column if x == 1 else f"inv_{column}")

processed_data['label_utilize'] = processed_data['label_utilize'].map({1: 'positive', 0: 'negative'})

headerless_data = processed_data.to_numpy()

correct_data_save_path = "/content/rrl/dataset/DATA.data"

np.savetxt(correct_data_save_path, headerless_data, fmt='%s', delimiter=",")

correct_data_save_path
num_features = headerless_data.shape[1] - 1
info_content = "\n".join([f"{i + 1} discrete" for i in range(num_features)])
info_content += "\nclass discrete\nLABEL_POS -1"

info_file_path = "/content/rrl/dataset/DATA.info"

with open(info_file_path, "w") as file:
    file.write(info_content)

info_file_path



In [ ]:
%cd /content/rrl

#RUN THE RRL MODEL

In [ ]:
!python3 /content/rrl/experiment.py -d /content/rrl/dataset/DATA -bs 16 -s 1@64 -e100 -lrde 200 -lr 0.01 -ki 0 -i 0 -wd 0.0001 --nlaf --alpha 0.8 --beta 3 --gamma 3 --temp 0.001 --print_rule &